#### 20220829, apply scale factor for corresonponding scripts.
- Propanal_C3H6O_PTR         = ACETONE_C3H6O_PTR.*0.17;
- Butanal_C4H8O_PTR          = MEK_C4H8O_PTR.*0.01;
- 2_Methylpropanal_C4H8O_PTR = MEK_C4H8O_PTR.*0.14
- 2_Methylbutanal_PTR        = 3_methyl_2_butanone_C5H10O_PTR.*0.04
- Ethyl_benzene_C8H10_PTR = C8_AROMATICS_C8H10_PTR.*0.36;
- mpXylenes_C8H10_PTR = C8_AROMATICS_C8H10_PTR.*0.46;
- oXylene_C8H10_PTR = C8_AROMATICS_C8H10_PTR.*0.18;
- acetic_acid_C2H4O2_PTR = acetic_acid_C2H4O2_PTR.*0.67;
- apinene_C10H16_PTR = MONOTERPENES_C10H16_PTR.*0.33;
- bpinene_C10H16_PTR = MONOTERPENES_C10H16_PTR.*0.21;
- MVK_C4H6O_PTR = MACR_MVK_C4H6O_PTR.*0.60;
- MACR_C4H6O_PTR = MACR_MVK_C4H6O_PTR.*0.28;
- ACETONE_C3H6O_PTR = ACETONE_C3H6O_PTR.*0.83;
- MEK_C4H8O_PTR = MEK_C4H8O_PTR.*0.85;
- 2_Methylfuran_C5H6O_PTR = 2_METHYLFURAN_C5H6O_PTR.*0.84;
- 3_Methylfuran_C5H6O_PTR = 2_METHYLFURAN_C5H6O_PTR.*0.16;
- 5_METHYLFURFURAL_C6H6O2_PTR = 5_METHYLFURFURAL_C6H6O2_PTR.*0.5;
- glycolaldehyde_C2H4O2_PTR = acetic_acid_C2H4O2_PTR.*0.33;
- hydroxyacetone_C3H6O2_PTR = hydroxyacetone_C3H6O2_PTR.*0.51;
- Methyl_acetate_C3H6O2_PTR = hydroxyacetone_C3H6O2_PTR.*0.35;
- Ethyl_formate_C3H6O2_PTR = hydroxyacetone_C3H6O2_PTR.*0.14;
- 2_Butenal_C4H6O_PTR = MACR_MVK_C4H6O_PTR.*0.13;
- iBuONO2_TOGA = iBuONO2and2BuONO2_TOGA.*0.5;
- nBuONO2_TOGA = iBuONO2and2BuONO2_TOGA.*0.5;
- MBO_C5H10O_PTR = 3_methyl_2_butanone_C5H10O_PTR.*0.43;
- 2_Pentanone_C5H10O_PTR = 3_methyl_2_butanone_C5H10O_PTR.*0.23;
- 3_Pentanone_C5H10O_PTR = 3_methyl_2_butanone_C5H10O_PTR.*0.21;
- Catechol_C6H6O2_PTR = 5_METHYLFURFURAL_C6H6O2_PTR.*0.5;
- Hexanones_C6H12O_PTR = 3_HEXANONE_C6H12O_PTR.*0.53;
- Hexanal_C6H12O_PTR = 3_HEXANONE_C6H12O_PTR.*0.47;
- heptanal_C7H14O_PTR = heptanal_C7H14O_PTR.*0.63;
- heptanone_C7H14O_PTR = heptanal_C7H14O_PTR.*0.37;

In [ ]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
import os
# Changing the CWD
os.chdir('/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_helper')
from F0AM_reader_MASTER import *

In [ ]:
#Read the full data
df_full = pd.read_csv('/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Model_inputs_prepared/raw_data/NEMRs.merged.transect.lixu_v3.csv',
                  sep=',') 
df_simplified = df_full.drop(['CO', 'Exact_mass', 'NEMR', 'Formula', 'C', 'H', 'O', 'N', 'Functional_group', 'K_OH', 'NMOG_contributor', 'In_OHR_work', 'Insturment', 'XS_ppb','LATITUDE', 'LONGITUDE', 'ALTP'], axis=1)
standard_col = 'plume_num'  # may need to be changed...
standard_col_uq = np.unique(df_simplified[standard_col])

# drop nan in standard column, ideally it will not work
standard_col_uq = standard_col_uq[~np.isnan(standard_col_uq)] 
df_simplified = df_simplified.dropna(subset=[standard_col])

data = []
for std_col in standard_col_uq:
    if std_col in ['137', '138', '139', '151', '152', '153', '154', '176', '178', '179']: continue # This is used to filter out Ferguson Fire in RF07
    
    df_std = df_simplified[df_simplified[standard_col] == std_col]

    # suppose to have use pressure measured by PTR/CIMS
    # These values should be constant for each plume number
    try:
        Flight_ID_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['Flight_ID'])[0]
        plume_num_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['plume_num'])[0]
        emission_pass_name_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['emission_pass_name'])[0]
        IsEmissionPass_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['IsEmissionPass'])[0]
        closest_fire_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['closest_fire'])[0]
        #SmkBkg_flg_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['SmkBkg_flg'])[0]
        Age_physical_avg_min_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['Age_physical_avg_min'])[0]

        pres_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['PRESSURE'])[0]
        temp_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['TEMPERATURE'])[0]
        RH_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['RHUM'])[0]
        SZA_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['SZA'])[0]
        JNO2_NO_O3P_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.NO2..NO.O.3P..'])[0]
        JHNO2_OH_NO_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.HNO2..OH.NO.'])[0]
        JH2O2_2OH_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.H2O2..2OH.'])[0]
        JHNO3_OH_NO2_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.HNO3..OH.NO2.'])[0]
        JO3_O2_O1D_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.O3..O2.O.1D..'])[0]
        JCH2O_H_HCO_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.CH2O..H.HCO.'])[0]
        JCH2O_H2_CO_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.CH2O..H2.CO.'])[0]
        JNO3_NO_O2_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.NO3..NO.O2.'])[0]
        JNO3_NO2_O_3P_assumption = list(df_std[df_std['species']=='m33.03341..METHANOL_CH3OH.H_cal...Corr._ppb']['j.NO3..NO2.O.3P..'])[0] # only 9 photolysis in this data, confirmed
    except:
        print('This is so wrong!!!')
        exit('This is wrong')
    # get needed data
    Flight_ID = Flight_ID_assumption
    plume_num = plume_num_assumption
    closest_fire = closest_fire_assumption
    emission_pass_name = emission_pass_name_assumption
    IsEmissionPass = IsEmissionPass_assumption
    #SmkBkg_flg = SmkBkg_flg_assumption
    Age_physical_avg_min = Age_physical_avg_min_assumption
    PRESSURE = pres_assumption
    TEMPERATURE = temp_assumption
    RHUM = RH_assumption
    SZA = SZA_assumption
    JNO2_NO_O3P = JNO2_NO_O3P_assumption
    JHNO2_OH_NO = JHNO2_OH_NO_assumption
    JH2O2_2OH = JH2O2_2OH_assumption
    JHNO3_OH_NO2 = JHNO3_OH_NO2_assumption
    JO3_O2_O1D = JO3_O2_O1D_assumption
    JCH2O_H_HCO = JCH2O_H_HCO_assumption
    JCH2O_H2_CO = JCH2O_H2_CO_assumption
    JNO3_NO_O2 = JNO3_NO_O2_assumption
    JNO3_NO2_O_3P = JNO3_NO2_O_3P_assumption

    species_list = df_std['species']
    conc_smk = df_std['conc_smk']
    conc_bkg = df_std['conc_bkg']

    # make the species list into more readable format
    species_list_new = []
    for ind, spec in enumerate(species_list):
        if '...' in spec:
            if 'dimethyl_furan_C6H8O' in spec:
                spec = '2_5_dimethyl_furan_C6H8O'
            elif '1.3.cyclopentadiene_C5H6' in spec:
                spec = '1_3_cyclopentadiene_C5H6'
            else:
                spec = spec.split('.')[3] + '_PTR'
        species_list_new.append(spec)
    species_list = np.array(species_list_new)

    conc_smk_spec = ['smk_' + x for x in species_list]
    conc_bkg_spec = ['bkg_' + x for x in species_list]
    conc_nemr_spec = ['nemr_' + x for x in species_list]

    # caclulate NEMR for each species
    ind = species_list == 'CO_QCL'

    CO_ehc = conc_smk[ind] - conc_bkg[ind]
    conc_nemr = (np.array(conc_smk)-np.array(conc_bkg))/np.array(CO_ehc) * 1000

    
    column_names = np.hstack(['Flight_ID', 'plume_num', 'closest_fire', 'emission_pass_name', 'IsEmissionPass', 'Age_physical_avg_min', 
                              'PRESSURE', 'TEMPERATURE', 'RHUM', 'SZA',
                              'JNO2_NO_O3P', 'JHNO2_OH_NO', 'JH2O2_2OH', 'JHNO3_OH_NO2', 'JO3_O2_O1D', 'JCH2O_H_HCO', 'JCH2O_H2_CO', 'JNO3_NO_O2', 'JNO3_NO2_O_3P', 
                              conc_smk_spec, conc_bkg_spec, conc_nemr_spec])
    data.append(np.hstack([Flight_ID, plume_num, closest_fire, emission_pass_name, IsEmissionPass, Age_physical_avg_min, 
                           PRESSURE, TEMPERATURE, RHUM, SZA,
                           JNO2_NO_O3P, JHNO2_OH_NO, JH2O2_2OH, JHNO3_OH_NO2, JO3_O2_O1D, JCH2O_H_HCO, JCH2O_H2_CO, JNO3_NO_O2, JNO3_NO2_O_3P,
                           conc_smk, conc_bkg, conc_nemr]))

df = pd.DataFrame(np.array(data), columns = column_names) # need to make panda understand we are putting rows


# soft conversion for data type
df["Age_physical_avg_min"] = pd.to_numeric(df["Age_physical_avg_min"])

In [3]:
df[df['Flight_ID']=='RF10'][['JNO2_NO_O3P', 'JHNO2_OH_NO', 'JH2O2_2OH', 'JHNO3_OH_NO2', 'JO3_O2_O1D', 'JCH2O_H_HCO', 'JCH2O_H2_CO', 'JNO3_NO_O2', 'JNO3_NO2_O_3P']]

,JNO2_NO_O3P,JHNO2_OH_NO,JH2O2_2OH,JHNO3_OH_NO2,JO3_O2_O1D,JCH2O_H_HCO,JCH2O_H2_CO,JNO3_NO_O2,JNO3_NO2_O_3P
149,0.002937031125,0.0003806371375,1.0094592625e-06,5.1544530000000005e-08,2.2681392500000002e-06,3.5348897499999997e-06,7.5278315e-06,0.0219790175,0.1636521125
150,0.00331419992857143,0.00045906113571428603,1.38561614285714e-06,7.76819021428571e-08,3.4619110714285697e-06,5.27957992857143e-06,1.05228698571429e-05,0.0202083285714286,0.15048248571428602
151,0.00452630484210526,0.000664845394736842,2.2685044210526298e-06,1.3835762631578902e-07,6.31540615789474e-06,9.23585831578947e-06,1.73736147368421e-05,0.0213763557894737,0.16255400526315802
152,0.00475615290909091,0.000629159027272727,1.69967945454545e-06,8.69757863636364e-08,3.7378959090909105e-06,6.05138718181818e-06,1.27853931818182e-05,0.0296645127272727,0.229029172727273
153,0.0030346816875,0.0004007586625,1.09256948125e-06,5.5380011875e-08,2.3191101875e-06,3.901014125e-06,8.2688201875e-06,0.022168095,0.163944675
154,0.0041638825,0.0006138915,2.046943125e-06,1.179811875e-07,5.011852625e-06,8.21021725e-06,1.593579875e-05,0.01996802,0.1502765125
155,0.007456614625,0.0011642493,4.45741875e-06,2.795351e-07,1.2090330750000001e-05,1.921874875e-05,3.4835818749999996e-05,0.02370022,0.19028475
156,0.0040693684,0.00060978194,2.1040442000000003e-06,1.2317974e-07,5.1715944e-06,8.6025272e-06,1.6438114e-05,0.015228976000000002,0.12094748
157,0.0097716675,0.0015575261666666699,6.3086195e-06,4.12019166666667e-07,1.81895566666667e-05,2.7812165e-05,4.9107101666666706e-05,0.026559885,0.217443066666667
158,0.0090834418,0.0014364273999999998,5.6572554e-06,3.5837918e-07,1.5335548e-05,2.4647244e-05,4.4320908e-05,0.025692566,0.21049562


In [4]:
# Edit: 20231016
# Read old data to get the j values
df_full_old = pd.read_csv('/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Model_inputs_prepared/raw_data/F0AM_inputs_all_fires_R0 copy.csv',sep=',')  
# derive j values
j_cols = ["in_plume"] + [col for col in df_full_old.columns if 'j[' in col]

df_full_old_j = df_full_old[j_cols]
# change the column name
df_full_old_j = df_full_old_j.rename(columns={"in_plume": "plume_num"}, inplace=False)
df_full_old_j['plume_num'] = df_full_old_j['plume_num'].astype(str)


In [5]:
df_full_old.columns

Index(['flight_id', 'closest_fire', 'in_plume', 'Age_physical_avg_min',
       'dist_from_fire_m', 'wind_speed_m_per_s', 'TEMPERATURE', 'PRESSURE',
       'SZA', 'RHUM',
       ...
       'j[BrO->Br+O]', 'j[HOBr->OH+Br]', 'j[BrNO->Br+NO]', 'j[BrONO->Br+NO2]',
       'j[BrONO->BrO+NO]', 'j[BrNO2->Br+NO2]', 'j[BrONO2->BrO+NO2]',
       'j[BrONO2->Br+NO3]', 'j[BrCl->Br+Cl]', 'j[CHBr3->Products]'],
      dtype='object', length=199)

In [6]:
df_full_old_j

,plume_num,j[O3->O2+O(1D)],j[NO2->NO+O(3P)],j[H2O2->2OH],j[NO3->NO+O2],j[NO3->NO2+O(3P)],j[N2O5->NO3+NO2],j[HNO2->OH+NO],j[HNO3->OH+NO2],j[HNO4->HO2+NO2(UV_VISonly)],...,j[BrO->Br+O],j[HOBr->OH+Br],j[BrNO->Br+NO],j[BrONO->Br+NO2],j[BrONO->BrO+NO],j[BrNO2->Br+NO2],j[BrONO2->BrO+NO2],j[BrONO2->Br+NO3],j[BrCl->Br+Cl],j[CHBr3->Products]
0,1,0.000043,0.011081,0.000009,0.024919,0.208370,0.000048,0.001810,7.685758e-07,0.000009,...,0.044622,0.002674,0.025920,0.013819,0.013819,0.005676,0.000252,0.001429,0.012923,1.495960e-06
1,2,0.000042,0.011079,0.000009,0.025315,0.211803,0.000047,0.001804,7.581774e-07,0.000008,...,0.044249,0.002675,0.026171,0.013691,0.013691,0.005692,0.000252,0.001428,0.012993,1.483836e-06
2,3,0.000041,0.011211,0.000009,0.025597,0.214603,0.000047,0.001821,7.507098e-07,0.000008,...,0.044351,0.002703,0.026524,0.013703,0.013703,0.005767,0.000254,0.001441,0.013183,1.468751e-06
3,4,0.000044,0.011504,0.000010,0.025427,0.212749,0.000050,0.001885,8.054640e-07,0.000009,...,0.046772,0.002771,0.026585,0.014488,0.014488,0.005871,0.000262,0.001483,0.013314,1.577280e-06
4,5,0.000039,0.011201,0.000009,0.026226,0.219977,0.000048,0.001807,7.274110e-07,0.000008,...,0.043611,0.002696,0.026866,0.013439,0.013439,0.005771,0.000253,0.001434,0.013258,1.460900e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,263,0.000019,0.007565,0.000005,0.022530,0.185276,0.000028,0.001159,3.781638e-07,0.000004,...,0.025743,0.001853,0.021091,0.007812,0.007812,0.004074,0.000170,0.000964,0.009790,7.836341e-07
251,264,0.000009,0.005259,0.000004,0.012807,0.106451,0.000017,0.000858,2.288129e-07,0.000003,...,0.020065,0.001282,0.013084,0.006068,0.006068,0.002759,0.000119,0.000672,0.006427,4.133303e-07
252,265,0.000009,0.006390,0.000004,0.018517,0.151490,0.000020,0.001001,2.383303e-07,0.000003,...,0.021833,0.001559,0.017512,0.006535,0.006535,0.003414,0.000142,0.000805,0.008206,4.530186e-07
253,266,0.000007,0.004672,0.000003,0.012873,0.105752,0.000015,0.000742,1.869550e-07,0.000002,...,0.016717,0.001142,0.012448,0.005023,0.005023,0.002487,0.000105,0.000593,0.005913,3.547712e-07


In [7]:

new_columns = []
for col in df_full_old_j.columns:
    col = col.replace('[', '_')
    col = col.replace(']', '')
    col = col.replace('->', '_')
    col = col.replace('+', '_')
    col = col.replace('(', '')
    col = col.replace(')', '')
    col = col.replace('j_', 'J')
    new_columns.append(col)
# Assigning the modified column names back to the dataframe
df_full_old_j.columns = new_columns
# find consistent plume num
plume_num_old = df_full_old_j['plume_num']
plume_num_new = df['plume_num']  # Assuming df2 is the name of your second dataframe
common_plume_num = list(set(plume_num_old) & set(plume_num_new))
common_plume_num = sorted(common_plume_num, key=int)


# apply to data to both old and new data
df = df[df['plume_num'].isin(common_plume_num)]
df = df.drop(columns=['JNO2_NO_O3P', 'JHNO2_OH_NO', 'JH2O2_2OH', 
                      'JHNO3_OH_NO2', 'JO3_O2_O1D', 'JCH2O_H_HCO', 'JCH2O_H2_CO', 'JNO3_NO_O2', 'JNO3_NO2_O_3P'], inplace=False) # drop repeated information
df_full_old_j = df_full_old_j[df_full_old_j['plume_num'].isin(common_plume_num)]
# combine the data together
combined_df = df.merge(df_full_old_j, on='plume_num', how='inner')
# Replace df with combined_df
df = combined_df

'''
# Remove outlier rows based on multiple conditions
# From dilution correceted measured data
df = df[~(((df['Flight_ID'] == 'RF03') & (df['Age_physical_avg_min'].isin([31.63849147, 136.2731999, 138.5187557]))) |
          ((df['Flight_ID'] == 'RF07') & (df['Age_physical_avg_min'].isin([141.05925190000005, 170.020107, 181.2327065]))) |
          ((df['Flight_ID'] == 'RF09') & (df['Age_physical_avg_min'].isin([58.72539609999995, 157.9950992, 158.72539609999995, 224.6374255, 236.0356894, 277.23208660000006]))))]

# From concentration
df = df[~(((df['Flight_ID'] == 'RF03') & (df['Age_physical_avg_min'].isin([23.5029077, 32.30082225, 132.4701964]))) |
          ((df['Flight_ID'] == 'RF07') & (df['Age_physical_avg_min'].isin([243.5556196]))) |
          ((df['Flight_ID'] == 'RF09') & (df['Age_physical_avg_min'].isin([ 158.72539609999995, 263.5098444]))))]


# From CO normalized data
df = df[~(((df['Flight_ID'] == 'RF03') & (df['Age_physical_avg_min'].isin([31.63849147, 136.2731999, 138.5187557]))) |
          ((df['Flight_ID'] == 'RF07') & (df['Age_physical_avg_min'].isin([ 121.4860684, 127.7832843, 141.05925190000005, 170.020107, 181.2327065, 276.8879582]))) |
          ((df['Flight_ID'] == 'RF09') & (df['Age_physical_avg_min'].isin([64.42733738, 157.9950992, 158.72539609999995, 197.50292030000003, 198.3964465, 224.6374255, 236.0356894, 277.23208660000006]))))]
'''
'''
# O3 vs TVOCR
#df = df[~(((df['Flight_ID'] == 'RF03') & (df['Age_physical_avg_min'].isin([23.5029077, 32.30082225, 132.47019640000002]))) |
#          ((df['Flight_ID'] == 'RF07') & (df['Age_physical_avg_min'].isin([243.5556196]))) |
#          ((df['Flight_ID'] == 'RF09') & (df['Age_physical_avg_min'].isin([ 158.72539609999995, 263.5098444]))))]
df = df[~(((df['Flight_ID'] == 'RF03') & (df['Age_physical_avg_min'].isin([23.5029077, 32.30082225, 124.7217351, 132.47019640000002]))) |
          ((df['Flight_ID'] == 'RF07') & (df['Age_physical_avg_min'].isin([157.768991, 243.5556196]))) |
          ((df['Flight_ID'] == 'RF09') & (df['Age_physical_avg_min'].isin([ 158.72539609999995, 263.5098444]))))]

# Based on NOx
df = df[~(((df['Flight_ID'] == 'RF03') & (df['Age_physical_avg_min'].isin([136.2731999]))) |
          ((df['Flight_ID'] == 'RF07') & (df['Age_physical_avg_min'].isin([141.05925190000005, 181.2327065, 276.8879582]))) |
          ((df['Flight_ID'] == 'RF09') & (df['Age_physical_avg_min'].isin([158.72539609999995, 236.0356894]))))]
'''

# Based on CO, check the figure, confirmed, really bad data
df = df[~(((df['Flight_ID'] == 'RF03') & (df['plume_num'].isin(['40', '50', '52','54', '56', '57']))))]
#df = df[~(((df['Flight_ID'] == 'RF07') & (df['plume_num'].isin(['140','141', '135','147', '139', '137' ,'142']))))]
#df = df[~(((df['Flight_ID'] == 'RF07') & (df['plume_num'].isin(['140','130', '139', '137' ,'142', '135', '133', '143', '147']))))]
df = df[~(((df['Flight_ID'] == 'RF07') & (df['plume_num'].isin(['140', '135', '137', '139', '142']))))]
df = df[~(((df['Flight_ID'] == 'RF09') & (df['plume_num'].isin(['161', '176','179', '154', '153']))))]

# Based on NOx, guess...
#df = df[~(((df['Flight_ID'] == 'RF03') & (df['plume_num'].isin(['44', '46', '49', '51', '53', '54', '55']))))]
#df = df[~(((df['Flight_ID'] == 'RF07') & (df['plume_num'].isin(['140', '135', '147', '139', '137', '136', '132', '146']))))]
#df = df[~(((df['Flight_ID'] == 'RF09') & (df['plume_num'].isin(['161', '158', '166', '169', '162', '179', '176', '164', '154', '178']))))]

# Based on O3 dil correction, guess...
#df = df[~(((df['Flight_ID'] == 'RF07') & (df['plume_num'].isin(['140', '135', '147', '139', '137', '136', '132', '146', '142']))))]


In [8]:
len(df[df['Flight_ID'] == 'RF10']['Age_physical_avg_min'])

1

In [9]:
df[df['Flight_ID'] == 'RF09'][['Age_physical_avg_min', 'plume_num', 'smk_NO', 'smk_NO2', 'smk_O3', 'bkg_O3', 'smk_CO_QCL', 'JHNO2_OH_NO', 'smk_BENZENE_C6H6_PTR']].sort_values(by='Age_physical_avg_min')

,Age_physical_avg_min,plume_num,smk_NO,smk_NO2,smk_O3,bkg_O3,smk_CO_QCL,JHNO2_OH_NO,smk_BENZENE_C6H6_PTR
126,24.569868,156,1.1271521428571398,5.13423571428571,73.9306428571429,56.264331975480395,1970.0142857142898,0.001460,3.44911336871638
127,47.139930,157,0.3977758333333329,2.3916833333333303,77.08825,55.494960999999996,1632.4666666666699,0.000877,2.76239041420717
137,64.427337,167,0.257178888888889,1.31150416666667,60.927194444444396,56.103908874335794,728.216666666667,0.001063,1.11938645881802
128,66.952290,158,0.18695681818181797,0.968487727272727,73.34536363636359,55.654335864135895,1021.60909090909,0.001117,1.61793559079436
129,89.560909,159,0.14323755,0.757679,70.72805,56.68271780538299,780.0,0.001527,1.19327439971052
138,106.827872,168,0.112738166666667,0.70409125,62.1229166666667,57.269092406476794,480.925,0.001138,0.676728443336427
130,113.689024,160,0.10710240909090898,0.585071818181818,69.4863636363636,57.052782592975205,690.854545454545,0.001586,1.03292832787632
136,123.450604,166,0.135014808823529,0.8108341176470591,67.6867352941176,56.534871049253,652.6705882352941,0.001180,0.955247894885681
139,154.956914,169,0.0518893461538462,0.394022307692308,67.62380769230771,58.034731036577995,542.192307692308,0.000800,0.7777315841074469
132,157.995099,162,0.0737460625,0.439105625,68.8531875,51.1933469187675,540.975,0.001472,0.6551320354976241


In [10]:
df[df['Flight_ID'] == 'RF07'][['Age_physical_avg_min', 'PRESSURE', 'TEMPERATURE', 'RHUM', 'SZA',
       'JO3_O2_O1D', 'JNO2_NO_O3P', 'JH2O2_2OH', 'JNO3_NO_O2', 'JNO3_NO2_O3P',
       'JN2O5_NO3_NO2', 'JHNO2_OH_NO', 'JHNO3_OH_NO2',
       'JHNO4_HO2_NO2UV_VISonly', 'JCH2O_H_HCO', 'JCH2O_H2_CO',
       'JCH3CHO_CH3_HCO', 'JPropanal_C2H5_HCO', 'JCH3OOH_CH3O_OH',
       'JMeONO2_CH3O_NO2', 'JEthONO2_CH3CH2O_NO2', 'JCH3COOONO2_CH3COOO_NO2',
       'JCH3COOONO2_CH3COO_NO3', 'JMAC_Products', 'JMVK_Products',
       'JAcetone_CH3CO_CH3', 'JMEK_CH3CO_CH2CH3', 'JHydroxyacetone_CH3COO_CH3',
       'JHydroxyacetone_CH3CO_CH3O', 'JCHOCHO_HCO_HCO', 'JCHOCHO_H2_2CO',
       'JCHOCHO_CH2O_CO', 'JCH3COCHO_CH3CO_HCO', 'J23Butanedione_Products']].sort_values(by='Age_physical_avg_min')

,Age_physical_avg_min,PRESSURE,TEMPERATURE,RHUM,SZA,JO3_O2_O1D,JNO2_NO_O3P,JH2O2_2OH,JNO3_NO_O2,JNO3_NO2_O3P,...,JMVK_Products,JAcetone_CH3CO_CH3,JMEK_CH3CO_CH2CH3,JHydroxyacetone_CH3COO_CH3,JHydroxyacetone_CH3CO_CH3O,JCHOCHO_HCO_HCO,JCHOCHO_H2_2CO,JCHOCHO_CH2O_CO,JCH3COCHO_CH3CO_HCO,J23Butanedione_Products
115,121.486068,574.7270752000001,275.5520807,17.727246,61.30510631,0.000010,0.008556,0.000005,0.029809,0.239381,...,0.000004,1.458459e-07,3.016931e-06,4.743552e-07,4.743552e-07,0.000067,0.000010,0.000020,0.000127,0.000335
105,127.783284,575.397972125,275.6414569,14.405685,33.14311423875,0.000042,0.011953,0.000009,0.027855,0.230388,...,0.000008,6.473122e-07,8.383687e-06,1.236365e-06,1.236365e-06,0.000097,0.000022,0.000039,0.000192,0.000387
109,134.104648,573.224873666667,275.5878454,24.3076185,41.255117958333294,0.000036,0.011783,0.000009,0.028970,0.239297,...,0.000007,5.563407e-07,7.474418e-06,1.106955e-06,1.106955e-06,0.000095,0.000020,0.000036,0.000186,0.000392
121,141.059252,574.592501545455,275.896470536364,9.023841000000001,66.7705407363636,0.000008,0.008440,0.000004,0.027984,0.226216,...,0.000004,1.120081e-07,2.686818e-06,4.329482e-07,4.329482e-07,0.000066,0.000010,0.000019,0.000124,0.000325
106,165.990953,575.332882473684,275.42184805789503,21.0469061578947,35.275518943684204,0.000013,0.007273,0.000004,0.036488,0.282809,...,0.000003,2.071637e-07,2.826615e-06,4.258886e-07,4.258886e-07,0.000058,0.000008,0.000016,0.000107,0.000327
120,188.552461,574.741358458333,275.167822066667,33.0666550833333,66.08913135125,0.000002,0.002779,0.000001,0.015175,0.111511,...,0.000001,2.862454e-08,7.045206e-07,1.147070e-07,1.147070e-07,0.000022,0.000003,0.000005,0.000040,0.000123
107,189.731528,575.5888671,274.87389410000003,27.387375899999995,36.662296085,0.000020,0.009089,0.000005,0.032846,0.262750,...,0.000005,3.159312e-07,4.297945e-06,6.437375e-07,6.437375e-07,0.000072,0.000012,0.000023,0.000138,0.000358
111,195.366879,572.981855142857,275.361779090476,19.6612776666667,43.30148660190479,0.000035,0.012411,0.000009,0.030041,0.250032,...,0.000008,5.330481e-07,7.521275e-06,1.117643e-06,1.117643e-06,0.000100,0.000021,0.000037,0.000195,0.000415
108,212.485758,575.300465916667,275.873327816667,35.4561366666667,38.6112178508333,0.000024,0.010343,0.000006,0.034612,0.279794,...,0.000005,3.711426e-07,5.109444e-06,7.650887e-07,7.650887e-07,0.000083,0.000014,0.000027,0.000158,0.000397
117,243.555620,574.5370482999999,274.9538936,16.387137,62.644693847,0.000014,0.010362,0.000006,0.027713,0.230815,...,0.000005,1.998217e-07,4.119754e-06,6.457417e-07,6.457417e-07,0.000081,0.000014,0.000026,0.000156,0.000369


In [11]:
# obsolete for now
def fix_PTR_isomers(df_conc, smk_conc=False, bkg_conc=False):
    setting = 'smk_' if smk_conc else 'bkg_'
    # Step 1: create a new species list
    data = pd.DataFrame()  # Placeholder for `ranking`
    data = pd.concat([data, 
                      df_conc[setting + 'ACETONE_C3H6O_PTR']*0.17,
                      df_conc[setting + 'MEK_C4H8O_PTR']*0.01,
                      df_conc[setting + 'MEK_C4H8O_PTR']*0.14,
                      df_conc[setting + '3_methyl_2_butanone_C5H10O_PTR']*0.04,
                      df_conc[setting + 'C8_AROMATICS_C8H10_PTR']*0.36,
                      df_conc[setting + 'C8_AROMATICS_C8H10_PTR']*0.46,
                      df_conc[setting + 'C8_AROMATICS_C8H10_PTR']*0.18,
                      df_conc[setting + 'acetic_acid_C2H4O2_PTR']*0.67,
                      df_conc[setting + 'MONOTERPENES_C10H16_PTR']*0.33,
                      df_conc[setting + 'MONOTERPENES_C10H16_PTR']*0.21,
                      df_conc[setting + 'MACR_MVK_C4H6O_PTR']*0.60,
                      df_conc[setting + 'MACR_MVK_C4H6O_PTR']*0.28,
                      df_conc[setting + 'ACETONE_C3H6O_PTR']*0.83,
                      df_conc[setting + 'MEK_C4H8O_PTR']*0.85,
                      df_conc[setting + '2_METHYLFURAN_C5H6O_PTR']*0.84,
                      df_conc[setting + '2_METHYLFURAN_C5H6O_PTR']*0.16,
                      df_conc[setting + '5_METHYLFURFURAL_C6H6O2_PTR']*0.5,     
                      df_conc[setting + 'acetic_acid_C2H4O2_PTR']*0.33,
                      df_conc[setting + 'hydroxyacetone_C3H6O2_PTR']*0.51,
                      df_conc[setting + 'hydroxyacetone_C3H6O2_PTR']*0.35,
                      df_conc[setting + 'hydroxyacetone_C3H6O2_PTR']*0.14,
                      df_conc[setting + 'MACR_MVK_C4H6O_PTR']*0.13,
                      df_conc[setting + 'iBuONO2and2BuONO2_TOGA']*0.5,
                      df_conc[setting + 'iBuONO2and2BuONO2_TOGA']*0.5,
                      df_conc[setting + '3_methyl_2_butanone_C5H10O_PTR']*0.43,
                      df_conc[setting + '3_methyl_2_butanone_C5H10O_PTR']*0.23,
                      df_conc[setting + '3_methyl_2_butanone_C5H10O_PTR']*0.21,
                      df_conc[setting + '5_METHYLFURFURAL_C6H6O2_PTR']*0.5,
                      df_conc[setting + '3_HEXANONE_C6H12O_PTR']*0.53,
                      df_conc[setting + '3_HEXANONE_C6H12O_PTR']*0.47, 
                      df_conc[setting + 'heptanal_C7H14O_PTR']*0.63,
                      df_conc[setting + 'heptanal_C7H14O_PTR']*0.37],axis=1)
    # Step 2: rename these columns
    col_names = [setting + 'Propanal_C3H6O_PTR', setting + 'Butanal_C4H8O_PTR', setting + '2_Methylpropanal_C4H8O_PTR', setting + '2_Methylbutanal_PTR',
                 setting + 'Ethyl_benzene_C8H10_PTR', setting + 'mpXylenes_C8H10_PTR', setting + 'oXylene_C8H10_PTR',
                 setting + 'acetic_acid_C2H4O2_PTR',
                 setting + 'apinene_C10H16_PTR', setting + 'bpinene_C10H16_PTR', setting + 'MVK_C4H6O_PTR', setting + 'MACR_C4H6O_PTR',
                 setting + 'ACETONE_C3H6O_PTR', setting + 'MEK_C4H8O_PTR',
                 setting + '2_Methylfuran_C5H6O_PTR', setting + '3_Methylfuran_C5H6O_PTR', setting + '5_METHYLFURFURAL_C6H6O2_PTR',
                 setting + 'glycolaldehyde_C2H4O2_PTR',
                 setting + 'hydroxyacetone_C3H6O2_PTR',
                 setting + 'Methyl_acetate_C3H6O2_PTR',
                 setting + 'Ethyl_formate_C3H6O2_PTR',
                 setting + '2_Butenal_C4H6O_PTR',
                 setting + 'iBuONO2_TOGA',
                 setting + 'nBuONO2_TOGA',
                 setting + 'MBO_C5H10O_PTR',
                 setting + '2_Pentanone_C5H10O_PTR',
                 setting + '3_Pentanone_C5H10O_PTR',
                 setting + 'Catechol_C6H6O2_PTR',
                 setting + 'Hexanones_C6H12O_PTR',
                 setting + 'Hexanal_C6H12O_PTR',
                 setting + 'heptanal_C7H14O_PTR',
                 setting + 'heptanone_C7H14O_PTR']

    data.columns = col_names
    # Step 3; remove the old used columns
    old_columns = list(set([setting + 'ACETONE_C3H6O_PTR',
                   setting + 'MEK_C4H8O_PTR',
                   setting + 'MEK_C4H8O_PTR',
                   setting + '3_methyl_2_butanone_C5H10O_PTR',
                   setting + 'C8_AROMATICS_C8H10_PTR',
                   setting + 'C8_AROMATICS_C8H10_PTR',
                   setting + 'C8_AROMATICS_C8H10_PTR',
                   setting + 'acetic_acid_C2H4O2_PTR',
                   setting + 'MONOTERPENES_C10H16_PTR',
                   setting + 'MONOTERPENES_C10H16_PTR',
                   setting + 'MACR_MVK_C4H6O_PTR',
                   setting + 'MACR_MVK_C4H6O_PTR',
                   setting + 'ACETONE_C3H6O_PTR',
                   setting + 'MEK_C4H8O_PTR',
                   setting + '2_METHYLFURAN_C5H6O_PTR',
                   setting + '2_METHYLFURAN_C5H6O_PTR',
                   setting + '5_METHYLFURFURAL_C6H6O2_PTR',
                   setting + 'acetic_acid_C2H4O2_PTR',
                   setting + 'hydroxyacetone_C3H6O2_PTR',
                   setting + 'hydroxyacetone_C3H6O2_PTR',
                   setting + 'hydroxyacetone_C3H6O2_PTR',
                   setting + 'MACR_MVK_C4H6O_PTR',
                   setting + 'iBuONO2and2BuONO2_TOGA',
                   setting + 'iBuONO2and2BuONO2_TOGA',
                   setting + '3_methyl_2_butanone_C5H10O_PTR',
                   setting + '3_methyl_2_butanone_C5H10O_PTR',
                   setting + '3_methyl_2_butanone_C5H10O_PTR',
                   setting + '5_METHYLFURFURAL_C6H6O2_PTR',
                   setting + '3_HEXANONE_C6H12O_PTR',
                   setting + '3_HEXANONE_C6H12O_PTR',
                   setting + 'heptanal_C7H14O_PTR',
                   setting + 'heptanal_C7H14O_PTR']))
    df_conc = df_conc.drop(old_columns, axis = 1)
    
    # Step 4: concat those two together
    return pd.concat([df_conc, data], axis = 1)
def WECAN_process(df, Flight_ID, smk_conc=False, bkg_conc=False, nemr_conc=False, met_data=False):
    """
    Processes and saves WE-CAN flight data based on specified parameters.

    Args:
        df (DataFrame): The dataframe containing the WE-CAN dataset.
        Flight_ID (str): The flight ID to process data for.
        smk_conc (bool): If True, processes smoke concentration data.
        bkg_conc (bool): If True, processes background concentration data.
        nemr_conc (bool): If True, processes NEMR concentration data.
        met_data (bool): If True, processes meteorological data.

    Returns:
        DataFrame: Processed and sorted dataframe based on specified parameters.
    """
    if smk_conc == True:
        output = df[df['Flight_ID'] == Flight_ID][['Age_physical_avg_min'] + conc_smk_spec]
        name = 'smk_conc'
    elif bkg_conc == True:
        output = df[df['Flight_ID'] == Flight_ID][['Age_physical_avg_min'] + conc_bkg_spec]
        name = 'bkg_conc'
    elif nemr_conc == True:
        output = df[df['Flight_ID'] == Flight_ID][['Age_physical_avg_min'] + conc_nemr_spec]
        name = 'nemr_conc'
    elif met_data == True:
        output = df[df['Flight_ID'] == Flight_ID][['Age_physical_avg_min', 'PRESSURE', 'TEMPERATURE', 'RHUM', 'SZA',
                                                  'JO3_O2_O1D', 'JNO2_NO_O3P','JH2O2_2OH', 'JNO3_NO_O2','JNO3_NO2_O3P','JN2O5_NO3_NO2','JHNO2_OH_NO',
                                                   'JHNO3_OH_NO2', 'JHNO4_HO2_NO2UV_VISonly','JCH2O_H_HCO','JCH2O_H2_CO','JCH3CHO_CH3_HCO','JPropanal_C2H5_HCO',
                                                   'JCH3OOH_CH3O_OH','JMeONO2_CH3O_NO2','JEthONO2_CH3CH2O_NO2','JCH3COOONO2_CH3COOO_NO2','JCH3COOONO2_CH3COO_NO3',
                                                   'JMAC_Products','JMVK_Products','JAcetone_CH3CO_CH3','JMEK_CH3CO_CH2CH3', 'JHydroxyacetone_CH3COO_CH3',
                                                   'JHydroxyacetone_CH3CO_CH3O','JCHOCHO_HCO_HCO','JCHOCHO_H2_2CO','JCHOCHO_CH2O_CO','JCH3COCHO_CH3CO_HCO',
                                                   'J23Butanedione_Products','JCl2_Cl_Cl','JClO_Cl_O3P','JClNO2_Cl_NO2','JClONO_Cl_NO2','JClONO2_Cl_NO3',
                                                   'JClONO2_ClO_NO2','JBr2_Br_Br','JBrO_Br_O','JHOBr_OH_Br','JBrNO_Br_NO','JBrONO_Br_NO2','JBrONO_BrO_NO',
                                                   'JBrNO2_Br_NO2','JBrONO2_BrO_NO2','JBrONO2_Br_NO3','JBrCl_Br_Cl','JCHBr3_Products']]
        name = 'met_data'
    # sort the data based on the pysical age
    output = output.sort_values(by="Age_physical_avg_min", ascending=True)
    
    # process missing values after sorted by physical age
    for col in output.columns:
        if col not in ['Flight_ID', 'plume_num', 'closest_fire', 'emission_pass_name', 'IsEmissionPass']:
            # assign them as float
            output[col] = output[col].astype('float')
            
            # replace negative value as 0
            output[col] = output[col].clip(lower=0)
            
            # replace 0 with nan
            output[col] = output[col].replace({0:np.nan})

            # linear regression for missing value
            output[col] = output[col].interpolate(method='linear', limit_direction='forward')
            output[col] = output[col].interpolate(method='backfill')
    
            # fill missing data with 0 back
            output[col] = output[col].replace({np.nan:0})

    # save data into csv
    output.to_csv('/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Model_inputs_prepared/output_data/' + Flight_ID + '_' + name + '.csv',
                  index=False)
    return output

# get matlab-like data from pandas dataframe
import scipy.io as sio

def get_F0AM_inputs(dataframe, Flight_ID, smk_conc=False, bkg_conc=False, nemr_conc=False, met_data=False):
    """
    Converts a pandas dataframe to a MATLAB-compatible format and saves it.

    Args:
        dataframe (DataFrame): The dataframe to be converted.
        Flight_ID (str): The flight ID corresponding to the data.
        smk_conc (bool): If True, processes smoke concentration data.
        bkg_conc (bool): If True, processes background concentration data.
        nemr_conc (bool): If True, processes NEMR concentration data.
        met_data (bool): If True, processes meteorological data.

    Saves:
        A .mat file containing the converted data in a specified directory.
    """
    if smk_conc == True:
        out_dir = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Model_inputs_prepared/output_data/mat/' + Flight_ID + '_smk.mat'
    elif bkg_conc == True:
         out_dir = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Model_inputs_prepared/output_data/mat/' + Flight_ID + '_bkg.mat'
    elif nemr_conc == True:
         out_dir = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Model_inputs_prepared/output_data/mat/' + Flight_ID + '_nemr.mat'
    elif met_data == True:
         out_dir = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Model_inputs_prepared/output_data/mat/' + Flight_ID + '_met.mat'

    data = {name: col.values for name, col in dataframe.items()}
    sio.savemat(out_dir, data)

def kdil(conc_smk, conc_bkg, time):
    """
    Calculates the dilution rate for smoke concentration data.

    Args:
        conc_smk (array): Array of smoke concentration values.
        conc_bkg (array): Array of background concentration values.
        time (array): Array of time values in minutes.

    Returns:
        array: Array of dilution rates for each time step.

    Note:
        If the calculated rate array is empty, returns an empty array.
    """
    # dX/dt = -kdil*(X - Xb)
    dXdt = np.diff(conc_smk)/np.diff(60*time)  # loss rate, sec

    # Check if dXdt is empty
    if dXdt.size > 0:
        dXdt = np.append(dXdt, dXdt[-1])  # extend the length to same length
    else:
        # Handle the case when dXdt is empty
        # You can choose to return an empty array, a specific value, or handle it another way
        return np.array([])

    res = -np.array(dXdt)/(np.array(conc_smk) - conc_bkg)
    return res

In [12]:
# get matlab-like data from pandas for conc, bkg, and met data
Flight_IDs = np.unique(df['Flight_ID']).astype('str')
Flight_IDs = ['RF01', 'RF02', 'RF03', 'RF04', 'RF05', 
              'RF06', 'RF07', 'RF09',
              'RF11', 'RF13']

for Flight_ID in Flight_IDs:
    # conc
    df_conc = WECAN_process(df, Flight_ID=Flight_ID, smk_conc=True)    
    #df_conc = fix_PTR_isomers(df_conc, smk_conc= True, bkg_conc=False)
    get_F0AM_inputs(df_conc, Flight_ID=Flight_ID, smk_conc=True)
    
    # bkg
    df_bkg = WECAN_process(df, Flight_ID=Flight_ID, bkg_conc=True)
    #df_bkg = fix_PTR_isomers(df_bkg, smk_conc= False, bkg_conc=True)
    get_F0AM_inputs(df_bkg, Flight_ID=Flight_ID, bkg_conc=True)
 
    # nemr
    #df_nemr = WECAN_process(df, Flight_ID=Flight_ID, nemr_conc=True)
    #get_F0AM_inputs(df_nemr, Flight_ID=Flight_ID, nemr_conc=True)
    
    # met
    # add integration time into the met data
    # calcualte the integration time step
    time = np.array(df_conc['Age_physical_avg_min'])
    
    # Setting start_ind, this is only for calculating InitTime
    # If we changed the start_ind here, please make sure the F0AM used the right datapoint to inialize the model.
    start_ind = 1 if (Flight_ID == 'RF03') else 0
    
    #print(time)
    
    # intTime
    InitTime = (float(time[-1])-float(time[start_ind]))*60/len(time)

    # Any dilution (constrained by CO and Benzene) less than 0 will be removed
    kdil_CO = kdil(df_conc['smk_CO_QCL'].astype('float'), df_bkg['bkg_CO_QCL'].astype('float'), time)
    kdil_benz = kdil(df_conc['smk_BENZENE_C6H6_PTR'].astype('float'), df_bkg['bkg_BENZENE_C6H6_PTR'].astype('float'), time)
    bool_index = (kdil_CO > 0) & (kdil_benz > 0)
    kdil_values = np.hstack([kdil_CO[bool_index], kdil_benz[bool_index]])
    kdil_med = np.median(kdil_values)
    df_met = WECAN_process(df, Flight_ID=Flight_ID, met_data=True)
    df_met['Int_time'] = [InitTime]*len(time)
    
    if Flight_ID == 'RF03': df_met['kdil'] = [3.5e-4]*len(time) # these kdils are from Qiaoyun
    if Flight_ID == 'RF07': df_met['kdil'] = [0.5e-4]*len(time)
    if Flight_ID == 'RF09': df_met['kdil'] = [1.75e-4]*len(time)#*10+[1e-4]*(len(time)-10)
    
    get_F0AM_inputs(df_met, Flight_ID=Flight_ID, met_data=True)
    print('This is the flight ID: ', Flight_ID)
    print('This is the integration time for the model', InitTime)
    print('This is the median dilution constant for model', kdil_med)


This is the flight ID:  RF01
This is the integration time for the model 1458.3402796499997
This is the median dilution constant for model 0.00025703154296509326
This is the flight ID:  RF02
This is the integration time for the model 2496.9135557739132
This is the median dilution constant for model 0.000840798770315901
This is the flight ID:  RF03
This is the integration time for the model 530.8989814285712
This is the median dilution constant for model 0.0004666155995617298
This is the flight ID:  RF04
This is the integration time for the model 1152.6782004600002
This is the median dilution constant for model 0.0004952908393634106
This is the flight ID:  RF05
This is the integration time for the model 1520.4880775294123
This is the median dilution constant for model 0.00016217199162389152
This is the flight ID:  RF06
This is the integration time for the model 1225.053130523077
This is the median dilution constant for model 0.0007005004688700753
This is the flight ID:  RF07
This is the 

In [13]:
conc_smk_spec

['smk_OA_AMS',
 'smk_Methane_PICARRO',
 'smk_Ethyne_AWAS',
 'smk_HCN_UWCIMS',
 'smk_HCN_PTR',
 'smk_HCN_TOGA',
 'smk_CO_QCL',
 'smk_ethene_C2H4_PTR',
 'smk_Ethene_AWAS',
 'smk_NO',
 'smk_formaldehyde_CH2O_PTR',
 'smk_CH2O_TOGA',
 'smk_Ethane_AWAS',
 'smk_METHANOL_CH3OH_PTR',
 'smk_CH3OH_TOGA',
 'smk_PROPYNE_C3H4_PTR',
 'smk_ACETONITRILE_C2H3N_PTR',
 'smk_CH3CN_TOGA',
 'smk_Acetonitrile_AWAS',
 'smk_propene_C3H6_PTR',
 'smk_Propylene_AWAS',
 'smk_isocyanic_acid_HNCO_PTR',
 'smk_ethenamine_C2H5N_PTR',
 'smk_CO2_PICARRO_ppb',
 'smk_NO2',
 'smk_ACETALDEHYDE_C2H4O_PTR',
 'smk_CH3CHO_TOGA',
 'smk_Acetaldehyde_AWAS',
 'smk_Propane_TOGA',
 'smk_Propane_AWAS',
 'smk_formamide_CH3NO_PTR',
 'smk_CH2O2_UWCIMS',
 'smk_formic_acid_CH2O2_PTR',
 'smk_ETHANOL_C2H6O_PTR',
 'smk_C2H5OH_TOGA',
 'smk_HONO_UWCIMS',
 'smk_HONO_HNO2_PTR',
 'smk_O3',
 'smk_CH3Cl_TOGA',
 'smk_1_buten_3_yne_C4H4_PTR',
 'smk_acrylonitrile_C3H3N_PTR',
 'smk_Acrylonitrile_TOGA',
 'smk_2_propynal_C3H2O_PTR',
 'smk_butadiene_C4H6_PTR

### Note (kdil)
RF03: 3.5e-4 is fine but a little bit high

RD07: 1e-4 seems fine

RF09: 1.9/2e-4 seems okay.

In [14]:
list(df_met.columns)

['Age_physical_avg_min',
 'PRESSURE',
 'TEMPERATURE',
 'RHUM',
 'SZA',
 'JO3_O2_O1D',
 'JNO2_NO_O3P',
 'JH2O2_2OH',
 'JNO3_NO_O2',
 'JNO3_NO2_O3P',
 'JN2O5_NO3_NO2',
 'JHNO2_OH_NO',
 'JHNO3_OH_NO2',
 'JHNO4_HO2_NO2UV_VISonly',
 'JCH2O_H_HCO',
 'JCH2O_H2_CO',
 'JCH3CHO_CH3_HCO',
 'JPropanal_C2H5_HCO',
 'JCH3OOH_CH3O_OH',
 'JMeONO2_CH3O_NO2',
 'JEthONO2_CH3CH2O_NO2',
 'JCH3COOONO2_CH3COOO_NO2',
 'JCH3COOONO2_CH3COO_NO3',
 'JMAC_Products',
 'JMVK_Products',
 'JAcetone_CH3CO_CH3',
 'JMEK_CH3CO_CH2CH3',
 'JHydroxyacetone_CH3COO_CH3',
 'JHydroxyacetone_CH3CO_CH3O',
 'JCHOCHO_HCO_HCO',
 'JCHOCHO_H2_2CO',
 'JCHOCHO_CH2O_CO',
 'JCH3COCHO_CH3CO_HCO',
 'J23Butanedione_Products',
 'JCl2_Cl_Cl',
 'JClO_Cl_O3P',
 'JClNO2_Cl_NO2',
 'JClONO_Cl_NO2',
 'JClONO2_Cl_NO3',
 'JClONO2_ClO_NO2',
 'JBr2_Br_Br',
 'JBrO_Br_O',
 'JHOBr_OH_Br',
 'JBrNO_Br_NO',
 'JBrONO_Br_NO2',
 'JBrONO_BrO_NO',
 'JBrNO2_Br_NO2',
 'JBrONO2_BrO_NO2',
 'JBrONO2_Br_NO3',
 'JBrCl_Br_Cl',
 'JCHBr3_Products',
 'Int_time']

In [15]:
list(df_bkg.columns)

['Age_physical_avg_min',
 'bkg_OA_AMS',
 'bkg_Methane_PICARRO',
 'bkg_Ethyne_AWAS',
 'bkg_HCN_UWCIMS',
 'bkg_HCN_PTR',
 'bkg_HCN_TOGA',
 'bkg_CO_QCL',
 'bkg_ethene_C2H4_PTR',
 'bkg_Ethene_AWAS',
 'bkg_NO',
 'bkg_formaldehyde_CH2O_PTR',
 'bkg_CH2O_TOGA',
 'bkg_Ethane_AWAS',
 'bkg_METHANOL_CH3OH_PTR',
 'bkg_CH3OH_TOGA',
 'bkg_PROPYNE_C3H4_PTR',
 'bkg_ACETONITRILE_C2H3N_PTR',
 'bkg_CH3CN_TOGA',
 'bkg_Acetonitrile_AWAS',
 'bkg_propene_C3H6_PTR',
 'bkg_Propylene_AWAS',
 'bkg_isocyanic_acid_HNCO_PTR',
 'bkg_ethenamine_C2H5N_PTR',
 'bkg_CO2_PICARRO_ppb',
 'bkg_NO2',
 'bkg_ACETALDEHYDE_C2H4O_PTR',
 'bkg_CH3CHO_TOGA',
 'bkg_Acetaldehyde_AWAS',
 'bkg_Propane_TOGA',
 'bkg_Propane_AWAS',
 'bkg_formamide_CH3NO_PTR',
 'bkg_CH2O2_UWCIMS',
 'bkg_formic_acid_CH2O2_PTR',
 'bkg_ETHANOL_C2H6O_PTR',
 'bkg_C2H5OH_TOGA',
 'bkg_HONO_UWCIMS',
 'bkg_HONO_HNO2_PTR',
 'bkg_O3',
 'bkg_CH3Cl_TOGA',
 'bkg_1_buten_3_yne_C4H4_PTR',
 'bkg_acrylonitrile_C3H3N_PTR',
 'bkg_Acrylonitrile_TOGA',
 'bkg_2_propynal_C3H2O_PTR',

In [16]:
df_conc['smk_CO_QCL']

170    2547.704545
169    3703.245455
168    2104.805556
167    1498.141176
166     811.707692
Name: smk_CO_QCL, dtype: float64

In [17]:
list(df_conc.columns)

['Age_physical_avg_min',
 'smk_OA_AMS',
 'smk_Methane_PICARRO',
 'smk_Ethyne_AWAS',
 'smk_HCN_UWCIMS',
 'smk_HCN_PTR',
 'smk_HCN_TOGA',
 'smk_CO_QCL',
 'smk_ethene_C2H4_PTR',
 'smk_Ethene_AWAS',
 'smk_NO',
 'smk_formaldehyde_CH2O_PTR',
 'smk_CH2O_TOGA',
 'smk_Ethane_AWAS',
 'smk_METHANOL_CH3OH_PTR',
 'smk_CH3OH_TOGA',
 'smk_PROPYNE_C3H4_PTR',
 'smk_ACETONITRILE_C2H3N_PTR',
 'smk_CH3CN_TOGA',
 'smk_Acetonitrile_AWAS',
 'smk_propene_C3H6_PTR',
 'smk_Propylene_AWAS',
 'smk_isocyanic_acid_HNCO_PTR',
 'smk_ethenamine_C2H5N_PTR',
 'smk_CO2_PICARRO_ppb',
 'smk_NO2',
 'smk_ACETALDEHYDE_C2H4O_PTR',
 'smk_CH3CHO_TOGA',
 'smk_Acetaldehyde_AWAS',
 'smk_Propane_TOGA',
 'smk_Propane_AWAS',
 'smk_formamide_CH3NO_PTR',
 'smk_CH2O2_UWCIMS',
 'smk_formic_acid_CH2O2_PTR',
 'smk_ETHANOL_C2H6O_PTR',
 'smk_C2H5OH_TOGA',
 'smk_HONO_UWCIMS',
 'smk_HONO_HNO2_PTR',
 'smk_O3',
 'smk_CH3Cl_TOGA',
 'smk_1_buten_3_yne_C4H4_PTR',
 'smk_acrylonitrile_C3H3N_PTR',
 'smk_Acrylonitrile_TOGA',
 'smk_2_propynal_C3H2O_PTR',

In [18]:
bool_index1 = df_full['Fractional_contribution'] != '1'
bool_index2 = df_full['Fractional_contribution'].notnull()
list(np.unique(df_full[bool_index1 & bool_index2]['NMOG_contributor']))

['1,3-Butadiene_0.98, 1,2-Butadiene_0.02',
 '2,3-Butanedione_0.95, Methyl acrylate_0.05',
 '2,5-Dimethylfuran_0.44, 2-Ethylfuran_0.1, C2-substituted furan isomers_0.46',
 '2-Furfural (=furaldehyde)_0.95, 3-Furfural_0.05',
 '2-Methylfuran_0.84, 3-Methylfuran_0.16',
 '2-Methylphenol (=o-cresol)_0.5, Anisol_0.5',
 '2-Methylpyridine_0.93, 3-Methylpyridine_0.07',
 '3-Furancarbonitrile_0.7, 2-Furancarbonitrile_0.3',
 '3-Methyl-2-butanone_0.43, 2-Methylbutanal, 3-Methylbutanal_0.04, 2-Pentanone_0.32, 3-Pentanone_0.21',
 '5-Methylfurfural_0.5, Benzene diols (=catechol, resorcinol)_0.5',
 'Acetic acid_0.67, glycolaldehyde (=hydroxyacetaldehyde)_0.33, Methyl formate_0',
 'Acetone_0.83, Propanal_0.17',
 'C2 Phenols_0.5, Methyl anisol_0.5',
 'C6 Diones_0.46, C6 1-DBE esters_0.54',
 'Camphene_0.38, a-Pinene_0.33, b-Pinene + Myrcene_0.21, Tricyclene_0.07',
 'Camphor_0.3, Oxygenated monoterpenes_0.7',
 'Cineole, Other oxygenated monoterpenes_0',
 'Cyclopentanone_0.33, 3-Methyl-3-buten-2-one_0.67',
 '